# 0. Imports

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.mode.chained_assignment = None  # default='warn'


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import matplotlib.pyplot as plt

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# 1. Hyper Parameters

In [ ]:
SAMPLES = 1000
EDA = True
TOP_TAGS = 100

# 2. Load Data

In [ ]:
os.system("echo Data loading: "+str(SAMPLES)+"...")

Le dataset porte sur plus de 2 millions d'images, pour accélérer la phase de développement et debug on va limiter la quantité de données chargée.

In [ ]:
if SAMPLES > 0:
    df_raw = pd.read_csv('/kaggle/input/safebooru/all_data.csv', nrows=SAMPLES)
else:
    df_raw = pd.read_csv('/kaggle/input/safebooru/all_data.csv')
df_raw

In [ ]:
df_raw.info()

In [ ]:
# Check thumbnail size : 150x113
#df_raw["preview_url"][101]

# 3. Feature Engineering

## 3.1 Feature Selection

In [ ]:
os.system("echo Feature Selection...")

Supression des colonnes inutile dans notre cas.

In [ ]:
useless = ["id", "created_at", "rating", "score", "sample_width", "sample_height", "preview_url"]
usefull = [i for i in df_raw.columns if i not in useless]
usefull

In [ ]:
df_clean = df_raw[usefull]
df_clean

## 3.2 Tags extraction

In [ ]:
os.system("echo Tags Extraction and Count...")

On extrait et on compte les occurences de chacun des tags présent dans la colonne *tags*. Par exemple en utilisant un mapping *dict* donnant pour chaque tag son occurence.

In [ ]:
if EDA:
    tags = dict()
    total = 0
    pct = 0
    for line in df_clean.tags:
        tokens = line.split()
        for token in tokens:
            if token in tags:
                tags[token] += 1
            else:
                tags[token] = 1
        total += 1
        if total % int(0.1*df_clean.shape[0]) == 0:
            pct += 10
            print(pct,"% extracted")
            os.system("echo "+str(pct)+"% extracted")
    print("There is",len(tags),"different tags")

Suppressions des tags à faible occurence (définit en hyperparamètre).

On ordonne nos tags en décroissant et on peut maintenant jeter un oeil aux tags les plus fréquents en produisant un histogramme. On peut extraire les élements de notre *dict* avec *items()* pour ordonner les pair key/value selon les valeurs.

In [ ]:
os.system("echo Tags Sort and Plot...")

In [ ]:
if EDA:
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Data to plot
    sorted_tags_count = sorted(tags.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)
    
    # Style
    sns.set(style="whitegrid")
    sns.set_color_codes("pastel")
    
    # Initialize the matplotlib figure
    f, ax = plt.subplots(figsize=(20, 50))

    # Chart
    sns.barplot(x=[j for (i,j) in sorted_tags_count[:TOP_TAGS]], y=[i for (i,j) in sorted_tags_count[:TOP_TAGS]], color='b', label="Occurences")

    # Legend and informative axis label
    ax.legend(ncol=1, loc="lower right", frameon=True)
    ax.set(ylabel="Tags",
           xlabel="Occurences")
    sns.despine(left=True, bottom=True)
    
    plt.show()

On observe dans les top tags *tagme* qui semble identifier les images sans tags et avec peu de tags. On a *solo* qui semble identifier des personnage seul. On voit aussi *1girl* qui peut indiquer une seule fille sur l'image, on peut s'attendre à avoir un tag *1boy*. On a le tag *male* qui doit identifier la présence de personnage masculin.

Aussi dans les tops tags on trouve *long_hair* et *short_hair* ça pourrait être une classification possible à apprendre à partir de ces données. De même pour les couleurs de cheveux et les type de vêtements. On observe aussi différents nom de série d'anime.

In [ ]:
if EDA:
    print("Occurences: ")
    selected_tags = ["solo", "1girl", "1boy", "girl", "female", "loli", "boy", "male", "man", "shota", "trap", "reverse_trap"]
    for tag in selected_tags:
        if tag in tags:
            print(tag, tags[tag])
        else:
            print(tag," ",0)

Les tags retenu pour identifier des personnages féminin ou masculin seul seront :
- tags 1 personnage :
    - solo
- tags féminins :
    - 1girl
    - girl
    - female
    - loli
- tags masculins :
    - 1boy
    - boy
    - male
    - man
    - shota
- tags trap :
    - trap
    - reverse_trap

On peut maintenant isoler les images contenant un seul personnage et dont le genre est connu. On va créer un nouvelle colonne *target* dans notre dataset *df_eng* suivant cet encodage:
* "?" indiquera le fait que l'image n'est pas identifiable aux tags présents (on peut pas déterminer si personnage seul ou le genre)
* "girl" indiquera une image de personnage féminin seule
* "boy" indiquera une image de personnage masculin seul

In [ ]:
os.system("echo Target creation...")

In [ ]:
def appears_in(l1,l2):
    for tag in l1:
        if tag in l2:
            return True
    return False

# Selected tags
solo_tags = ["solo"]#, "1girl", "1boy"]
girl_tags = ["1girl", "girl", "female", "loli"]
boy_tags = ["1boy", "boy", "male", "man" "shota"]
trap_tags = ["trap","reverse_trap"]

# Tags detections
tagged_solo = [appears_in(solo_tags,i.split())  for i in df_clean.tags]
tagged_girl = [appears_in(girl_tags,i.split()) for i in df_clean.tags]
tagged_boy = [appears_in(boy_tags,i.split()) for i in df_clean.tags]
tagged_trap = [appears_in(trap_tags,i.split()) for i in df_clean.tags]

# Target conversion
target = []

for i in range(0,len(df_clean)):
    value = "?" # encode not valid sample
    if tagged_solo[i] and (tagged_girl[i] != tagged_boy[i]) and (not tagged_trap[i]):
        if tagged_girl[i]:
            value = "girl" # 1 encode girl sample
        else:
            value = "boy" # 2 encode boy sample
    target.append(value)

df_clean["target"] = target
df_clean

Suppression des images non exploitables et de colonnes devenu inutiles

In [ ]:
os.system("echo Cleaning non class image...")

In [ ]:
df_clean = df_clean[df_clean.target != "?"]
df_clean.drop(['tags'], axis=1, inplace=True)
df_clean

In [ ]:
print("Reduced dataset:")
print("Number of images:", df_clean.shape[0])
print("Number of girls:", df_clean[df_clean.target==1].shape[0])
print("Number of boys:", df_clean[df_clean.target==2].shape[0])

In [ ]:
os.system("echo Reduced dataset:")
os.system("echo Number of images:"+str(df_clean.shape[0]))
os.system("echo Number of girls:"+str(df_clean[df_clean.target==1].shape[0]))
os.system("echo Number of boys:"+str(df_clean[df_clean.target==2].shape[0]))

## 3.3 Images Extraction

In [ ]:
os.system("echo Downloading "+str(df_clean.shape[0])+ " images...")

On va devoir récupérer les images directement sur les serveurs de *sanbooru.org* à partir des urls contenu dans le dataframe pour consituer notre dataset d'image en local.

On commence par créer le dossier *images* où seront stocker nos images.

In [ ]:
!mkdir "images"
!ls

On créer une fonction permettant de télécharger une image depuis une *url* et qui la stock dans *images/*. Une façon de faire est d'utiliser *request* comme dans https://www.quickprogrammingtips.com/python/how-to-download-multiple-files-concurrently-in-python.html

In [ ]:
import requests
 
def download_url(url):
  #print("downloading: ",url)
  # assumes that the last segment after the / represents the file name
  # if url is abc/xyz/file.txt, the file name will be file.txt
  file_name_start_pos = url.rfind("/") + 1
  file_name = "images/" + url[file_name_start_pos:]
 
  r = requests.get(url, stream=True)
  if r.status_code == requests.codes.ok:
    with open(file_name, 'wb') as f:
      for data in r:
        f.write(data)
  return file_name

On récupère la liste de toutes les url d'images depuis le dataframe *df_clean*.

In [ ]:
urls = ["http:"+i for i in df_clean.sample_url]

On utilise notre fonction *download_url* pour chaque image. On pourra utiliser *ThreadPool* pour faire un téléchargement groupé. En effet si les serveur de *sanbooru* on moins d'upload qu'on a de download on perdra du temps à les faire 1 par 1.

In [ ]:
from multiprocessing.pool import ThreadPool
pd.options.mode.chained_assignment = None

# Run 5 multiple threads. Each call will take the next element in urls list
result = ThreadPool(5).imap_unordered(download_url, urls)
total = 0
pct =0
for r in result:
    total += 1
    if total % int(0.01*df_clean.shape[0]) == 0:
        pct = round(100/df_clean.shape[0] * total)
        print(pct,"% downloaded")
        os.system("echo "+str(pct)+"% downloaded")
    if total % 1000 == 0:
        print(total,"/",df_clean.shape[0]," downloaded")
        os.system("echo "+str(total)+"/"+str(df_clean.shape[0])+" downloaded")
        pct = round(100/df_clean.shape[0] * total)
        os.system("echo "+str(pct)+"% downloaded")
        

On peut maintenant créer la nouvelle colonne et supprimer *sample_url*. *urls* nous assure l'ordre des images dans le dataset et puisqu'on utilise la fin d'url comme nom de fichier on peut la récupérer en tronquant de la même façon.

In [ ]:
images_path = [url[url.rfind("/") + 1:] for url in urls]
images_path[:10]

On remplace la colonne *sample_url* par une nouvelle colonne donnant le path des images à présent téléchargées.

In [ ]:
df_clean["file"] = images_path
df_clean.drop(['sample_url'], axis=1, inplace=True)
df_clean

Suppression des images dont le download a échoué.

In [ ]:
os.system("echo Delete failed downloaded entries...")

In [ ]:
size_before = df_clean.shape[0]
for index, row in df_clean.iterrows():
    if not os.path.isfile("images/"+str(row["file"])):
        df_clean.drop(index, inplace=True)
df_clean

In [ ]:
os.system("echo Failed Download: "+str(df_clean.shape[0]-size_before))
print("echo Failed Download: "+str(size_before-df_clean.shape[0]))

Suppressions des images corrompues

In [ ]:
from PIL import Image
size_before = df_clean.shape[0]
for index, row in df_clean.iterrows():
    try:
        img = Image.open("images/"+row["file"]) # open the image file
    except (IOError, SyntaxError) as e:
        print('Bad file:', "images/"+row["file"]) # print out the names of corrupt files
        os.remove("images/"+row["file"])
        df_clean.drop(index, inplace=True)
df_clean

In [ ]:
from os import listdir
from PIL import Image

bad_count = 0
for directory in ["images/"]:
    for filename in listdir(directory):
        try:
            img = Image.open(directory+filename) # open the image file
        except (IOError, SyntaxError) as e:
            print('Bad file:', directory+filename) # print out the names of corrupt files
            os.remove(directory+filename)
            df_clean.drop()
            bad_count += 1
    print(bad_count," error files")

Plus qu'a afficher un petit expample des image récupérées.

Un peu stylisé avec du ipywidget.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define a useful function
def get_image(f_path):
    '''
    Returns the image from a path
    '''
    img_labs = ['jpg','png']
    if any(x in img_labs for x in f_path.split('.')):
        image = open(f_path,'rb').read()
        return image

In [ ]:
some_girls = df_clean[df_clean.target == 1].head(20).file

images = [get_image("images/"+x) for x in some_girls]
children = [widgets.Image(value = img) for img in images if str(type(img)) != '<class \'NoneType\'>']
labels = ['{}'.format(i) for i in range(len(children))]

# Customize your layout here:
box_layout = widgets.Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    border='solid',
    width='50%')

# Create the widget
tab = widgets.Tab()
tab.children = children

# Label em'!
for i in range(len(children)):
    tab.set_title(i,labels[i])

display(tab)

In [ ]:
some_boys = df_clean[df_clean.target == 2].head(20).file

images = [get_image("images/"+x) for x in some_boys]
children = [widgets.Image(value = img) for img in images if str(type(img)) != '<class \'NoneType\'>']
labels = ['{}'.format(i) for i in range(len(children))]

# Customize your layout here:
box_layout = widgets.Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    border='solid',
    width='25%')

# Create the widget
tab = widgets.Tab()
tab.children = children

# Label em'!
for i in range(len(children)):
    tab.set_title(i,labels[i])

display(tab)

# 4. Dataset output

In [ ]:
os.system("echo Save to csv...")

Sauvegarde du dataframe en csv.

In [ ]:
df_clean.to_csv("sanboru_gender_dataset.csv", index=False)

Regroupement des images en un seul fichier zip.

In [ ]:
os.system("echo Zip images...")

In [ ]:
import shutil
shutil.make_archive("sanboru_gender_dataset", 'zip', "images")
!rm images/*
!rm -d "images"

Affichage de nos fichiers créés.

In [ ]:
!ls

In [ ]:
os.system("echo Generated dataset: sanboru_gender_dataset.zip")
os.system("echo Number of images:"+str(df_clean.shape[0]))
os.system("echo Number of girls:"+str(df_clean[df_clean.target==1].shape[0]))
os.system("echo Number of boys:"+str(df_clean[df_clean.target==2].shape[0]))

In [ ]:
print("Generated dataset: sanboru_gender_dataset.zip")
print("Number of images:", df_clean.shape[0])
print("Number of girls:", df_clean[df_clean.target==1].shape[0])
print("Number of boys:", df_clean[df_clean.target==2].shape[0])